In [ ]:
import sys
sys.path.insert(0, '../lsdviztools') # remove when updated on main 
import glob
import lsdviztools.lsdmapwrappers as lsdmw
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
from IPython.utils import io

import parsl
from parsl import python_app
from parsl.config import Config
from parsl.channels import LocalChannel
from parsl.executors import HighThroughputExecutor
from parsl.providers import LocalProvider

import csv
from pathlib import Path
import pandas as pd
from osgeo import gdal_array
import numpy as np
import geopandas as gpd

# import seaborn as sns
import matplotlib.pyplot as plt

import seaborn as sns
matplotlib.rcParams.update({"pdf.fonttype":42})
%matplotlib inline

If you load in the results after you ran through once you don't have to do it again (I think?)

In [ ]:
shed_list = pd.read_csv("shed_list_for_dd_new.csv", dtype = {'HYBAS_ID':'int', 'DD':'float64'},
                        #  index_col='HYBAS_ID'
                         )
from pathlib import Path
p = Path('./lsdtt/')
id_list = [f.name for f in p.iterdir() if f.is_dir()]

# Run LSDTT algorithms on your new DEMs, extract and save data

## Do  lsdtt channel extraction algorithm

In [ ]:
htex_local = Config(
    executors = [
        HighThroughputExecutor(
            max_workers = 15, # Caps the number of workers launched per node
            provider = LocalProvider(
                worker_init="module load conda; conda activate lsdtopy",
                max_blocks = 1
            )
        )
    ],
)
parsl.clear()
parsl.load(htex_local)


In [ ]:
@python_app
def run_lsdtt(id):
    import sys
    sys.path.insert(0, '../lsdviztools') # remove when updated on main 
    from pathlib import Path
    import glob
    import lsdviztools.lsdmapwrappers as lsdmw
    from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
    from IPython.utils import io

    surface_fitting_radius = '100'
    m_over_n = '0.5'    

    p = Path(f'./lsdtt/{id}/')
    
    if (p / f'{id}_UTM_mn_0.5_D_CN.csv').exists() == True:
        return f'{id} already processed'

    else:
        print(f'Now working on {id}')
        DataDirectory = f'./lsdtt/{id}/'
        RasterFile = f'{id}.tif'
    try:
        gio.convert4lsdtt(DataDirectory, RasterFile,minimum_elevation=0.1,resolution=10)

        lsdtt_parameters = {
                            # Table 3
                            "carve_before_fill" : "true", # in case there are dams, 
                            "raster_is_filled" : "false",
                            # Table 4
                            "print_area_threshold_channels" : "true",
                            "print_dreich_channels" : "true",
                            # Table 5
                            # "write_hillshade" : "true",
                            "print_channels_to_csv" : "true",
                            # "print_curvature_raster" : "true", # doesn't work
                            "print_tangential_curvature": "true",
                            # Table 6
                            "surface_fitting_radius" : surface_fitting_radius, 
                            "threshold_contributing_pixels" : '5000', # Equivalent to threshold from MERIT for 10m DEM
                            "A_0" : '1.0', # Default 1.0, used by DrEICH
                            #### Change the m/n!
                            # "m_over_n" : 0.5, # Default 0.5, used by DrEICH
                            "m_over_n" : m_over_n, # Default 0.5, used by DrEICH
                            #####
                            # Table 7
                            "print_dinf_drainage_area_raster" : "true",
                            # "print_d8_drainage_area_raster" : "true",
                            "pruning_drainage_area" : "10",
                            "connected_components_threshold" : "10"
                            
                            }
        r_prefix = f'{id}_UTM'
        w_prefix = f'{id}_{surface_fitting_radius}m_mn_{m_over_n}'
        lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-channel-extraction", 
                                        read_prefix = r_prefix,
                                        write_prefix= w_prefix,
                                        read_path = f'./lsdtt/{id}/',
                                        write_path = f'./lsdtt/{id}/',
                                        parameter_dictionary=lsdtt_parameters)
        lsdtt_drive.print_parameters()
        lsdtt_drive.run_lsdtt_command_line_tool()
        
        return id
    except AttributeError: # Maybe some data are weird bc "'NoneType' object has no attribute 'GetProjection'"
       return f'{id} is bad'


In [ ]:
all_app_futures = []

for id in id_list:
    app_future = run_lsdtt(id)
    all_app_futures.append(app_future)

# By getting the `result()` of each app future, this block won't continue to 
# the print statement until all the files are staged.
[app_future.result() for app_future in all_app_futures] 

print("All IDs have been processed.")

In [ ]:
# Shutdown and clear the parsl executor
htex_local.executors[0].shutdown()
parsl.clear()

## Analyze lsdtt data

In [ ]:
surface_fitting_radius = '100'
m_over_n = '0.5'

In [ ]:
p = Path('./lsdtt/')
id_list = [f.name for f in p.iterdir() if f.is_dir()]

### Calculate drainage density 

In [ ]:
data_list = []
no_bils = []
no_data = []
no_channels = []



for id in id_list:
    w_prefix = f'{id}_{surface_fitting_radius}m_mn_{m_over_n}'

    AT_CN = f'{w_prefix}_AT_CN.csv'
    D_CN = f'{w_prefix}_D_CN.csv'
    dinf =  f'{w_prefix}_dinf_area.bil'
    # tan_curv = f'{id}_UTM_{surface_fitting_radius}m_TANCURV.bil'

    # d8 =  f'{w_prefix}_d8_area.bil'

    filepath = (p / f'{id}' / f'{dinf}')

    try: 
        # Read raster data as numeric array from file
        rasterArray = gdal_array.LoadFile(filepath.as_posix())
        #Create a masked array for making calculations without nodata values
        rasterArray = np.ma.masked_equal(rasterArray, -9999.).compressed()

        rastersize = len(rasterArray)

        if rastersize != 0:
            with open((p / f'{id}' / f'{AT_CN}').as_posix(),"r") as f:
                reader = csv.reader(f,delimiter = ",")
                data = list(reader)
                AT_CN_count = len(data) - 1 

            try:    
                with open((p / f'{id}' / f'{D_CN}').as_posix(),"r") as f:
                    reader = csv.reader(f,delimiter = ",")
                    data = list(reader)
                    D_CN_count = len(data) - 1
            except FileNotFoundError:
                print(f'No DrEICH channels in {id}')
                D_CN_count = np.nan
                no_channels.append([id])

            data = [id, (AT_CN_count/rastersize), (D_CN_count/rastersize)]
        else:
            print(f'Raster of size 0 for {id}')
            no_data.append([id])
            data = [id,np.nan,np.nan]

    except ValueError:
        print(f'No .bil made for for {id}')
        data = [id,np.nan,np.nan]
        no_bils.append([id])
    
    data_list.append(data)


df = pd.DataFrame(data=data_list, columns=['HYBAS_ID', 'AT_CN_DD', 'D_CN_DD'])

In [ ]:
fields = ['HYBAS_ID']

with open('no_channels.csv', 'w') as f:
  # using csv.writer method from CSV package
  write = csv.writer(f)
    
  write.writerow(fields)
  write.writerows(no_channels)

with open('no_bils.csv', 'w') as f:
  # using csv.writer method from CSV package
  write = csv.writer(f)
    
  write.writerow(fields)
  write.writerows(no_bils)

In [ ]:
shed_list['HYBAS_ID'] = shed_list['HYBAS_ID'].astype(str)

merged = shed_list.merge(df.dropna(), on="HYBAS_ID")

merged.to_csv("lsdtt_results.csv")

# Analyze results

In [ ]:
merged = pd.read_csv("lsdtt_results.csv")

In [ ]:
perm = merged[merged['EXTENT'] != 'No permafrost']
noperm = merged[merged['EXTENT'] == 'No permafrost']
continuous = merged[merged['EXTENT']=='Continuous']

In [ ]:
fig, ax = plt.subplots()

im1 = ax.hexbin(merged['DD'],merged['D_CN_DD'], gridsize=(70,50), mincnt=1, cmap='plasma')
im0 = sns.regplot(x='DD', y='D_CN_DD',  fit_reg=True,
                  scatter_kws={'alpha':0.0,
                              's':5},
                  ax=ax,
                  data=noperm,
                  line_kws={'color':'k'},
                 color='k')
cb = fig.colorbar(im1)
cb.set_label("Watershed counts in hex bin")
ax.set_xlim(0.09,0.20)
ax.set_ylim(0,0.075)
ax.set_xlabel("Hydrography90m channel drainage density (pix/pix)")
ax.set_ylabel("DrEICH algorithm channel drainage density (pix/pix)")
plt.savefig('./figure_outputs/supp_DD_method_comparison.png', bbox_inches="tight")

In [ ]:
import scipy as sp
r, p = sp.stats.pearsonr(merged['DD'], merged['D_CN_DD'])

print(r,p)

In [ ]:
import statsmodels.api as sm
re = sm.OLS(merged['D_CN_DD'], merged['DD']).fit()
print(re.summary())
merged['residual_DD'] = re.resid


In [ ]:
merged.groupby(['EXTENT'])['residual_DD'].mean()

## flow accumulation distributions

In [ ]:
hist_dict = {}

p = Path(f'./lsdtt/')

surface_fitting_radius = '100'
m_over_n = '0.5'  

for id in id_list:
    w_prefix = f'{id}_{surface_fitting_radius}m_mn_{m_over_n}'

    dinf =  f'{w_prefix}_dinf_area.bil'

    p = Path(f'./lsdtt/')

    filepath = (p / f'{id}' / f'{dinf}')

    try: 
        # Read raster data as numeric array from file
        rasterArray = gdal_array.LoadFile(filepath.as_posix())
        #Create a masked array for making calculations without nodata values
        rasterArray = np.ma.masked_equal(rasterArray, -9999.).compressed()

        hist, edges = np.histogram(rasterArray, np.logspace(2,7,70))

        hist_dict[id] = hist

    except ValueError:
        print(f'No .bil made for for {id}')
        # data = [id,np.nan,np.nan]
        # no_bils.append([id])

In [ ]:
perm_list = []
noperm_list = []
c_list = []

for id in noperm['HYBAS_ID'].astype(str):
    if id in hist_dict:
        arr = hist_dict[id]/hist_dict[id].sum()
        # exceedence = np.cumsum(arr[::-1])[::-1]
        # noperm_list.append(exceedence)
        cad = np.cumsum(arr)
        noperm_list.append(cad)

for id in perm['HYBAS_ID'].astype(str):
    if id in hist_dict:
        arr = hist_dict[id]/hist_dict[id].sum()
        # exceedence = np.cumsum(arr[::-1])[::-1]
        # perm_list.append(exceedence)
        cad = np.cumsum(arr)
        perm_list.append(cad)              
                        

for id in continuous['HYBAS_ID'].astype(str):
    if id in hist_dict:
        arr = hist_dict[id]/hist_dict[id].sum()
        # exceedence = np.cumsum(arr[::-1])[::-1]
        # c_list.append(exceedence)
        cad = np.cumsum(arr)
        c_list.append(cad)

# New heatmap

In [ ]:
arctic = gpd.read_file('../arctic_h90.shp').drop_duplicates(subset='HYBAS_ID', keep="first")
midlat = gpd.read_file('../midlat_h90.shp').drop_duplicates(subset='HYBAS_ID', keep="first")
joined = pd.concat([arctic, midlat]).drop_duplicates(subset='HYBAS_ID', keep="first")
joined = joined.set_crs("EPSG:4326").to_crs('EPSG:5936')
# Dumb, move this
joined['DD'] = joined['segment']/joined['flow_acc']

In [ ]:
joined['HYBAS_ID'] = joined['HYBAS_ID'].astype(str)

In [ ]:
merged['HYBAS_ID'] = merged['HYBAS_ID'].astype(str)

In [ ]:
atts = merged.merge(joined, on="HYBAS_ID")

permafrost = atts[atts['EXTENT'] == 'Continuous']
nonpermafrost = atts[atts['EXTENT'] == 'No permafrost']

In [ ]:
pts=5
map_max=700
relief_max=1200
map_array=np.append(np.linspace(-1,map_max,num=pts), [np.inf])
relief_array=np.append(np.linspace(-1,relief_max,num=pts), [np.inf])

nonpermafrost['bin_map'] = pd.cut(nonpermafrost['bio12_mean'],bins=map_array, labels=np.arange(len(map_array)-1))
nonpermafrost['bin_relief'] = pd.cut(nonpermafrost['relief'],bins=relief_array, labels=np.arange(len(relief_array)-1))
nonpermafrost_out = nonpermafrost.groupby(['bin_map','bin_relief'])['D_CN_DD']

nonpermafrost_counts = nonpermafrost_out.count().sort_index().values.reshape(len(relief_array)-1,len(map_array)-1)

nonpermafrost_arr = nonpermafrost_out.mean().sort_index().values.reshape(len(relief_array)-1,len(map_array)-1)

permafrost['bin_map'] = pd.cut(permafrost['bio12_mean'],bins=map_array, labels=np.arange(len(map_array)-1),
                             )
permafrost['bin_relief'] = pd.cut(permafrost['relief'],bins=relief_array, labels=np.arange(len(relief_array)-1),
                              )
permafrost_out = permafrost.groupby(['bin_map','bin_relief'])['D_CN_DD']

permafrost_counts = permafrost_out.count().sort_index().values.reshape(len(relief_array)-1,len(map_array)-1)

permafrost_arr = permafrost_out.mean().sort_index().values.reshape(len(relief_array)-1,len(map_array)-1)

In [ ]:
import scipy
import itertools
p = np.empty([pts,pts])
p[:] = np.nan
for i in itertools.product(range(pts),range(pts)):
    try:
        U1, p[i] = scipy.stats.mannwhitneyu(
        permafrost_out.get_group(i),
        nonpermafrost_out.get_group(i))
    except KeyError:
        pass

In [ ]:
shed_count_thresh = 0

pm1 = np.ma.masked_less(p, 1.00e-04)
pm1_masked = np.ma.masked_where(permafrost_counts < shed_count_thresh, pm1)
pm1_masked = np.ma.masked_where(nonpermafrost_counts < shed_count_thresh, pm1_masked)


pm2 = np.ma.masked_inside(p, 1.00e-04, 1.00e-03)
pm2_masked = np.ma.masked_where(permafrost_counts < shed_count_thresh, pm2)
pm2_masked = np.ma.masked_where(nonpermafrost_counts < shed_count_thresh, pm2_masked)

ratio = permafrost_arr/nonpermafrost_arr
ratio_masked = np.ma.masked_where(permafrost_counts < shed_count_thresh, ratio)
ratio_masked = np.ma.masked_where(nonpermafrost_counts < shed_count_thresh, ratio_masked)

In [ ]:
counts = permafrost_counts+nonpermafrost_counts
counts_masked = np.ma.masked_where(permafrost_counts == 0, counts)
counts_masked = np.ma.masked_where(nonpermafrost_counts == 0, counts_masked)

In [ ]:
np.sum(counts)

In [ ]:
np.sum(counts_masked)

In [ ]:
from matplotlib import colors

fig, ax = plt.subplots(3,1,figsize=(3,3),dpi=300, sharey=True, sharex=True)

im0 = ax[0].pcolor(ratio, cmap='PRGn', 
               #extent=[np.min(relief_array), np.max(relief_array), np.min(map_array), np.max(map_array)],
                vmin=0.5, vmax=1.5,
                   )

# im1 = ax[1].pcolor(permafrost_counts+nonpermafrost_counts)

im2 = ax[1].pcolor(pm2_masked,
                   norm=colors.LogNorm(vmin=1e-4,vmax=1e-1,),
                   
                  #   hatch='//',
                  #   alpha=0
                    )

im3 = ax[2].pcolor(counts_masked,
                  #  norm=colors.LogNorm()
                  #   hatch='//',
                  #   alpha=0
                  cmap='magma'
                    )


cb0 = fig.colorbar(im0)
cb0.set_label("Drainage density\nratio (permafrost\n/nonpermafrost)", fontsize=6)
cb2 = fig.colorbar(im2)
cb2.set_label("p value,\nMann-Whitney U", fontsize=6)
cb3 = fig.colorbar(im3)
cb3.set_label("Watershed counts\nin bin (total permafrost\n+nonpermafrost)", fontsize=6)

ax[0].set_xticks([0,2.5,5])
ax[0].set_xticklabels(['0',str(relief_max/2),'>'+str(relief_max)])

ax[0].set_yticks([0,2.5,5])
ax[0].set_yticklabels(['0',str(map_max/2),'>'+str(map_max)])

fig.tight_layout()
ax[2].set_xlabel("Relief (m)")
ax[0].set_ylabel("MAP (mm/yr)")
plt.savefig('./figure_outputs/supp_dreich_ratios.png', bbox_inches="tight")

# Now get curvature

## This is just a one time thing so I don't rerun everything

In [ ]:
htex_local = Config(
    executors = [
        HighThroughputExecutor(
            max_workers = 10, # Caps the number of workers launched per node
            provider = LocalProvider(
                worker_init="module load conda; conda activate lsdtopy",
                max_blocks = 1
            )
        )
    ],
)
parsl.clear()
parsl.load(htex_local)

In [ ]:
@python_app
def get_curve_and_jct_lsdtt(id):
    import sys
    sys.path.insert(0, '../lsdviztools') # remove when updated on main 
    from pathlib import Path
    import glob
    import lsdviztools.lsdmapwrappers as lsdmw
    from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
    from IPython.utils import io

    surface_fitting_radius = '100'
    m_over_n = '0.5'    

    p = Path(f'./lsdtt/{id}/')
    
    print(f'Now working on {id}')

    try:
        # gio.convert4lsdtt(DataDirectory, RasterFile,minimum_elevation=0.1,resolution=10)

        basic_parameters = {
                                # Table 3
                                "carve_before_fill" : "true", # in case there are dams, 
                                "raster_is_filled" : "false",
                                # Table 4
                                # Table 5
                                # "write_hillshade" : "true",
                                # "print_channels_to_csv" : "true",
                                "print_curvature" : "true",
                                "print_tangential_curvature": "true",
                                # Table 6
                                "surface_fitting_radius" : surface_fitting_radius, 
                                "threshold_contributing_pixels" : '500', # Equivalent to threshold from Hydrography90m for 10m DEM
                                "A_0" : '1.0', # Default 1.0, used by DrEICH
                                #### Change the m/n!
                                # "m_over_n" : 0.5, # Default 0.5, used by DrEICH
                                "m_over_n" : m_over_n, # Default 0.5, used by DrEICH
                                #####

                                ## more secrets

                                # "print_junction_angles_to_csv" : "true",
                                # "print_junction_angles_to_csv_in_basins": "true",
                                # "print_area_threshold_channels" : "true",

                                }


        r_prefix = f'{id}_UTM'
        w_prefix = f'{id}_{surface_fitting_radius}m_mn_{m_over_n}'

        # basic metrics does junction angle and curvature for some reason 

        lsdtt_drive = lsdmw.lsdtt_driver(
                                        # command_line_tool = "lsdtt-channel-extraction",
                                        command_line_tool = "lsdtt-basic-metrics",
                                        read_prefix = r_prefix,
                                        write_prefix= f'{w_prefix}',
                                        read_path = f'./lsdtt/{id}/',
                                        write_path = f'./lsdtt/{id}/',
                                        parameter_dictionary=basic_parameters)

        lsdtt_drive.print_parameters()
        lsdtt_drive.run_lsdtt_command_line_tool()

        # lsdtt_drive = lsdmw.lsdtt_driver(
        #                                 command_line_tool = "lsdtt-channel-extraction",
        #                                 # command_line_tool = "lsdtt-basic-metrics",
        #                                 read_prefix = r_prefix,
        #                                 write_prefix= w_prefix,
        #                                 read_path = f'./lsdtt/{id}/',
        #                                 parameter_dictionary=channel_parameters)

        # lsdtt_drive.print_parameters()
        # lsdtt_drive.run_lsdtt_command_line_tool()
        
        return id
    except AttributeError: # Maybe some data are weird bc "'NoneType' object has no attribute 'GetProjection'"
       return f'{id} is bad'


In [ ]:
all_app_futures = []

for id in id_list:
    app_future = get_curve_and_jct_lsdtt(id)
    all_app_futures.append(app_future)

# By getting the `result()` of each app future, this block won't continue to 
# the print statement until all the files are staged.
[app_future.result() for app_future in all_app_futures] 

print("All IDs have been processed.")

In [ ]:
# Shutdown and clear the parsl executor
htex_local.executors[0].shutdown()
parsl.clear()

# Curvature area plots

In [ ]:
def curve_area(id):
    
    from scipy.stats import binned_statistic

    w_prefix = f'{id}_{surface_fitting_radius}m_mn_{m_over_n}'

    dinf_file =  f'{w_prefix}_dinf_area.bil'

    curvature_file =  f'{w_prefix}_500px_for_junction_angles_TANCURV.bil'

    p = Path(f'./lsdtt/')

    dinf_filepath = (p / f'{id}' / f'{dinf_file}')

    curvature_filepath = (p / f'{id}' / f'{curvature_file}')

    try: 
        # Read raster data as numeric array from file
        dinf = gdal_array.LoadFile(dinf_filepath.as_posix())
        #Create a masked array for making calculations 
        dinf = np.ma.masked_equal(dinf, -9999.).flatten()
        # Read raster data as numeric array from file
        curvature = gdal_array.LoadFile(curvature_filepath.as_posix())
        #Create a masked array for making calculations 
        curvature = np.ma.masked_equal(curvature, -9999.).flatten()

        bin_edges = np.logspace(2,7,70)

        med_stat_ca = binned_statistic(dinf,curvature,
                                statistic='median',
                                bins=bin_edges)
        
        stats_dict[id] = med_stat_ca.statistic


    except ValueError:
        print(f'No .bil made for for {id}')
        # data = [id,np.nan,np.nan]
        # no_bils.append([id])






In [ ]:
stats_dict = {}

p = Path(f'./lsdtt/')

id_list = [f.name for f in p.iterdir() if f.is_dir()]

for id in id_list:
    med_stat_ca = curve_area(id)

look, this is criminal and i dont care

In [ ]:
len(stats_dict)

In [ ]:
merged = pd.read_csv("lsdtt_results.csv")
merged['HYBAS_ID'] = merged['HYBAS_ID'].astype(str)
merged.dtypes

In [ ]:
noperm = merged[merged['EXTENT'] == 'No permafrost']
continuous = merged[merged['EXTENT']=='Continuous']

perm_list_c = []
noperm_list_c = []
c_list_c = []

for id in noperm['HYBAS_ID']:
    if id in stats_dict:
        arr = stats_dict[id]
        noperm_list_c.append(arr)

for id in continuous['HYBAS_ID']:
    if id in stats_dict:
        arr = stats_dict[id]
        c_list_c.append(arr)

In [ ]:
noperm_list_c_scaled = np.nanmedian(noperm_list_c, axis=0) * 1e5
c_list_c_scaled = np.nanmedian(c_list_c, axis=0) * 1e5

In [ ]:
import matplotlib
fig, ax = plt.subplots(2,1,
                       sharex=True,
                       sharey=True,
                        figsize=(3.4252,3.4252),dpi=300,
                        gridspec_kw={'height_ratios':[1,0.8]})
# for i, arr in enumerate(perm_list):
#     ax.plot(np.logspace(2,7,70)[:-1], perm_list[i], 'b', alpha=0.03)
# for i,arr in enumerate(noperm_list):    
#     ax.plot(np.logspace(2,7,70)[:-1], noperm_list[i], 'r', alpha=0.03)
# ax.fill_between(np.logspace(2,7,70)[:-1],
#                 np.mean(c_list, axis=0)-np.std(c_list, axis=0),
#                 np.mean(c_list, axis=0)+np.std(c_list, axis=0),
#                 color='gray', alpha=0.1)
# ax.fill_between(np.logspace(2,7,70)[:-1],
#                 np.mean(noperm_list, axis=0)-np.std(noperm_list, axis=0),
#                 np.mean(noperm_list, axis=0)+np.std(noperm_list, axis=0),
#                 color='darkgray', alpha=0.1)
ax[0].plot(np.logspace(2,7,70)[:-1], np.median(c_list, axis=0), 'lightsteelblue', linewidth=2, zorder=1)
ax[1].plot(np.logspace(2,7,70)[:-1], np.median(noperm_list, axis=0), 'lightcoral', linewidth=2, zorder=1)

ax[1].scatter(np.logspace(2,7,70)[:-1], np.median(noperm_list, axis=0), s=20, c=noperm_list_c_scaled, zorder=2, vmin=-10, vmax=10, cmap='BrBG',
            edgecolor='lightcoral',
              linewidth=0.5,
            )
forthecolors = ax[0].scatter(np.logspace(2,7,70)[:-1], np.median(c_list, axis=0), s=20, c=c_list_c_scaled, zorder=2, vmin=-10, vmax=10, cmap='BrBG',
                           edgecolor='lightsteelblue',
                             linewidth=0.5,
                           )

fmt = matplotlib.ticker.ScalarFormatter(useMathText=True)
cb0 = fig.colorbar(forthecolors, location="top", format=fmt)
cb0.set_label("Curvature (m$^-$$^1$) x 10$^-$$^5$")


ax[0].axvline(np.interp(0, np.nanmedian(c_list_c, axis=0), np.logspace(2,7,70)[:-1]), 0, 0.8, color = 'lightsteelblue', linestyle=":", zorder=0)
ax[1].axvline(np.interp(0, np.nanmedian(noperm_list_c, axis=0), np.logspace(2,7,70)[:-1]), 0.70, 1, color='lightcoral', linestyle=':', zorder=0)


ax[0].set_xlim(9e1, 1e5)
ax[0].set_ylim(0, 1)
ax[0].set_xscale("log")

ax[1].set_xlabel("Drainage area (m$^2$) of pixels")
plt.ylabel("Cumulative area distribution", y=1)
plt.savefig("./figure_outputs/main_curvatures.pdf", bbox_inches="tight")

# Stats

At what drainage area do you get zero curvature?

In [ ]:
np.interp(0, np.nanmedian(c_list_c, axis=0), np.logspace(2,7,70)[:-1])

In [ ]:
np.interp(0, np.nanmedian(noperm_list_c, axis=0), np.logspace(2,7,70)[:-1])

At what drainage area do you get "high" curvature (1e-4)?

In [ ]:
np.interp(1e-4, np.nanmedian(c_list_c, axis=0), np.logspace(2,7,70)[:-1])

In [ ]:
np.interp(1e-4, np.nanmedian(noperm_list_c, axis=0), np.logspace(2,7,70)[:-1])

What percent of the landscape is composed of negative curvature values?

In [ ]:
np.nanmedian(c_list, axis=0)[np.argmin(abs(np.nanmedian(c_list_c, axis=0)))]

In [ ]:
np.nanmedian(noperm_list, axis=0)[np.argmin(abs(np.nanmedian(noperm_list_c, axis=0)))]

How many are there?

In [ ]:
len(noperm_list)

In [ ]:
len(c_list)

# Supplement

In [ ]:
supplement_warm_shed = '7100377840'

In [ ]:
nonpermafrost.loc[nonpermafrost['HYBAS_ID'] == supplement_warm_shed].columns

In [ ]:
nonpermafrost.loc[nonpermafrost['HYBAS_ID'] == supplement_warm_shed].values

In [ ]:
supplement_cold_shed = '3100413670'
# permafrost.loc[(permafrost['bin_map'] == 2) & (permafrost['bin_relief'] == 2)].sample(1)

In [ ]:
permafrost.loc[permafrost['HYBAS_ID'] == supplement_cold_shed]['D_CN_DD']